## Data Preparation

In [1]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from pymongo import MongoClient
import json, re
from re import sub
from decimal import Decimal
import dask.dataframe as dd
from dask.distributed import Client, progress
import dask.dataframe as dd
from dask.delayed import delayed

### Dask Client

In [2]:
client = Client(n_workers=4, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:33455 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.26 GB


### Database Connection

In [3]:
DB_USER = 'anutibara'
DB_PASS = 'anutibara'
DB_HOST = 'scraping-cluster-7dtgt.gcp.mongodb.net'
DB_NAME = 'scraping_db'

try:
    client = MongoClient(f'mongodb+srv://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}')
    print("Database connected successfully")
except Exception as e:
    print("Error to connect to database: ", e)
db = client.get_database(DB_NAME)
properties = db.properties

Database connected successfully


### Database Queries

In [4]:
properties.count_documents({})

14209

### Convert Mongo Collection to DataFrame

#### New Properties

In [5]:
new_properties_json = list(properties.find({ 'use': 'Nuevo' }))

In [6]:
df_general_info = json_normalize(new_properties_json, record_path='offersType', 
                                meta=['_id', 
                                    'urlProperty',
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [7]:
df_location = json_normalize(new_properties_json, record_path='location', meta='urlProperty')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [8]:
df_owner_property = json_normalize(new_properties_json, record_path='ownerProperty', meta='urlProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [9]:
df_features = json_normalize(new_properties_json, record_path='features', meta='urlProperty')
ddf_features = dd.from_pandas(df_features, npartitions=10)
new_columns = ['admon_price', 'antiquity', 'general_bathrooms', 'condition', 'construction_area',
               'floor', 'garages', 'includes_administration', 'interior_floors', 'range_prices', 
               'range_private_area', 'general_rooms', 'square_meters', 'square_meters_price', 'stratum', 
               'weather', 'urlProperty']
ddf_features = ddf_features.rename(columns=dict(zip(ddf_features.columns, new_columns)))

In [10]:
df_more_features = json_normalize(new_properties_json, record_path='moreFeatures', meta='urlProperty')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [11]:
ddf_list = [ddf_general_info, ddf_location, ddf_owner_property, ddf_features, ddf_more_features]
for ddf in ddf_list:
    ddf.set_index('urlProperty')

ddf = dd.concat(ddf_list, axis=1)
ddf.reset_index()

/home/stiven/anaconda3/lib/python3.7/site-packages/distributed/worker.py:3165: UserWarning: Large object of size 2.90 MB detected in task graph: 
  (         area bathrooms offerType        price pr ...  'urlProperty')
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


,index,area,bathrooms,offerType,price,privateArea,property,rooms,_id,urlProperty,scrapingDate,scrapingHour,modifyDate,modifyHour,code,status,type,use,nameProject,description,address,city,country,department,latitude,longitude,neighborhood,sector,urlProperty,contractType,financing,id,name,schedule,urlProperty,admon_price,antiquity,general_bathrooms,condition,construction_area,floor,garages,includes_administration,interior_floors,range_prices,range_private_area,general_rooms,square_meters,square_meters_price,stratum,weather,urlProperty,exteriorFeatures,interiorFeatures,sectorFeatures,urlProperty
npartitions=21,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,object,object,object,object,object,object,object,object,object,float64,float64,object,object,object,object,object,int64,object,object,object,float64,object,int64,object,object,object,object,object,int64,object,object,int64,object,float64,object,object,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
ddf_new_properties = ddf.loc[:,~ddf.columns.duplicated()]

#### Old Properties

In [13]:
old_properties_json = list(properties.find({ 'use': 'Usado' }))

In [14]:
df_general_info = pd.DataFrame(old_properties_json, 
                                    columns=['_id', 
                                    'urlProperty', 
                                    'scrapingDate', 
                                    'scrapingHour', 
                                    'modifyDate',
                                    'modifyHour', 
                                    'code', 
                                    'status', 
                                    'type', 
                                    'use', 
                                    'nameProject', 
                                    'description'])
ddf_general_info = dd.from_pandas(df_general_info, npartitions=10)

In [15]:
df_location = json_normalize(old_properties_json, 'location')
ddf_location = dd.from_pandas(df_location, npartitions=10)

In [16]:
df_owner_property = json_normalize(old_properties_json, 'ownerProperty')
ddf_owner_property = dd.from_pandas(df_owner_property, npartitions=10)

In [17]:
df_features = json_normalize(old_properties_json, 'features')
ddf_features = dd.from_pandas(df_features, npartitions=10)

In [18]:
df_more_features = json_normalize(old_properties_json, 'moreFeatures')
ddf_more_features = dd.from_pandas(df_more_features, npartitions=10)

In [19]:
ddf_list = [ddf_general_info, ddf_location, ddf_owner_property, ddf_features, ddf_more_features]
ddf = dd.concat(ddf_list, axis=1)
ddf.reset_index()

,index,_id,urlProperty,scrapingDate,scrapingHour,modifyDate,modifyHour,code,status,type,use,nameProject,description,address,city,country,department,latitude,longitude,neighborhood,sector,contractType,financing,id,name,schedule,admonPrice,antiquity,bathrooms,condition,constructionArea,floor,garages,includesAdministration,interiorFloors,price,privateArea,rooms,squareMeters,squareMetersPrice,stratum,weather,exteriorFeatures,interiorFeatures,sectorFeatures
npartitions=20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,object,object,object,object,object,object,int64,object,object,object,object,object,object,object,object,object,float64,float64,object,object,object,object,int64,object,object,float64,object,int64,object,object,object,object,object,int64,object,object,int64,object,float64,object,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [20]:
ddf_old_properties = ddf.loc[:,~ddf.columns.duplicated()]

### Rename DataFrame Columns

#### New Properties

In [21]:
new_columns = ['area', 'bathrooms', 'offer_type', 'price', 'private_area',
               'property', 'rooms', 'id_mongoose', 'id_property', 'scraping_date', 
               'scraping_hour', 'modify_hour', 'modify_date', 'code', 'active', 'type',
               'new_property', 'name_project', 'description', 'address', 'city', 'country', 
               'department', 'latitude', 'longitude', 'neighborhood', 'sector', 'contract_type',
               'financing', 'id', 'name', 'schedule', 'admon_price', 'antiquity', 'general_bathrooms',
               'condition', 'construction_area', 'floor', 'garages', 'includes_administration', 
               'interior_floors', 'range_prices', 'range_private_area', 'general_rooms', 'square_meters',
               'square_meters_price', 'stratum', 'weather', 'exterior_features', 'interior_features', 
               'sector_features']
ddf_new_properties = ddf_new_properties.rename(columns=dict(zip(ddf_new_properties.columns, new_columns)))

#### Old Properties

In [22]:
new_columns = ['id_mongoose', 'id_property', 'scraping_date', 
               'scraping_hour', 'modify_hour', 'modify_date', 'code', 'active', 'type',
               'new_property', 'name_project', 'description', 'address', 'city', 'country', 
               'department', 'latitude', 'longitude', 'neighborhood', 'sector', 'contract_type',
               'financing', 'id', 'name', 'schedule', 'admon_price', 'antiquity', 'general_bathrooms',
               'condition', 'construction_area', 'floor', 'garages', 'includes_administration', 
               'interior_floors', 'range_prices', 'range_private_area', 'general_rooms', 'square_meters',
               'square_meters_price', 'stratum', 'weather', 'exterior_features', 'interior_features', 
               'sector_features']
ddf_old_properties = ddf_old_properties.rename(columns=dict(zip(ddf_old_properties.columns, new_columns)))

### Data Cleaning

#### New Properties

In [27]:
ddf_new_properties['active'] = ddf_new_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_new_properties['new_property'] = ddf_new_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_new_properties['includes_administration'] = ddf_new_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_new_properties['garages'] = ddf_new_properties['garages'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['stratum'] = ddf_new_properties['stratum'].replace('', value = 0, regex = True).astype(int)
ddf_new_properties['floor'] = ddf_new_properties['floor'].astype(int)
ddf_new_properties['area'] = ddf_new_properties['area'].astype(float)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].replace('', value = 0, regex = True)
ddf_new_properties['private_area'] = ddf_new_properties['private_area'].astype(float)
ddf_new_properties['rooms'] = ddf_new_properties['rooms'].astype(int)
ddf_new_properties['bathrooms'] = ddf_new_properties['bathrooms'].astype(int)
ddf_new_properties['price'] = ddf_new_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_new_properties['price'] = ddf_new_properties['price'].astype(int)

#### Old Properties

In [31]:
ddf_old_properties['active'] = ddf_old_properties['active'].apply(lambda status: True if (status == 'Active') else False, meta=('active', 'bool'))
ddf_old_properties['new_property'] = ddf_old_properties['new_property'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('new_property', 'bool'))
ddf_old_properties['includes_administration'] = ddf_old_properties['includes_administration'].apply(lambda status: True if (status == 'Nuevo') else False, meta=('includes_administration', 'bool'))
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].replace('Más de 10', value = 0, regex = True)
ddf_old_properties['garages'] = ddf_old_properties['garages'].astype(int)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].replace('Campestre', value = 0, regex = True)
ddf_old_properties['stratum'] = ddf_old_properties['stratum'].astype(int)
ddf_old_properties['price'] = ddf_old_properties['price'].apply(lambda price: Decimal(sub(r'[^\d,]', '', price)), meta=('price', 'int'))
ddf_old_properties['price'] = ddf_old_properties['price'].astype(int)
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].str[0:-3]
ddf_old_properties['square_meters'] = ddf_old_properties['square_meters'].apply(lambda meters: Decimal(sub(r'[^\d,]', '', meters))).astype(float)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].str[0:-2]
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].replace('', value = '0', regex = True)
ddf_old_properties['private_area'] = ddf_old_properties['private_area'].apply(lambda area: Decimal(sub(r'[^\d.]', '', area))).astype(float)
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].str[0:-3]
ddf_old_properties['construction_area'] = ddf_old_properties['construction_area'].apply(lambda area: Decimal(sub(r'[^\d,]', '', area))).astype(float)
ddf_old_properties['floor'] = ddf_old_properties['floor'].astype(int)

KeyError: 'price'

### Testing

#### New Properties

In [ ]:
ddf_new_properties

#### Old Properties

In [ ]:
ddf_old_properties